In [1]:
using Distributed

In [2]:
procs = addprocs(8)

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [3]:
length(procs)

8

In [4]:
@everywhere begin

const d = 4
const N = 480
const P = Int(floor(2*N/3)) #N
const nbits = max(53, Int(floor(P*log2(10))))

# choose a binary precision matching P as closely as we can.
setprecision(BigFloat, nbits)

const c_text = "0.5754"
const r_text = "0.8"
    
const c = BigFloat(c_text)
const r = BigFloat(r_text)

prefix = "feig_d$(d)_N$(N)_P$(P)_c$(c_text)_r$(r_text)_"
    
end

d, N, P, nbits, c, r

(4, 480, 320, 1063, 0.575400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001, 0.799999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996)

In [5]:
@everywhere begin

import Base: <, <=, ==, >, >=
import Base: zero, one
import Base: abs
import Base: +, -, *, /, ^

struct Poly
    coeffs::Array{BigFloat, 1}
    cr::Tuple{BigFloat, BigFloat}
    Poly(a::Array{BigFloat, 1}, cr) = length(a) == 1+N ? new(a, cr) : error("wrong #coeffs")
end

abs(t::Poly) = sum(abs(a) for a in t.coeffs)
zero(p::Poly) = Poly(fill(BigFloat(0), N+1), p.cr)
one(p::Poly) = Poly([k==0 ? BigFloat(1) : BigFloat(0) for k in 0:N], p.cr)

(-)(p::Poly) = Poly(-p.coeffs, p.cr)

(*)(p::Poly, a::Number) = Poly(p.coeffs*a, p.cr)
(*)(a::Number, p::Poly) = p*a

(/)(p::Poly, a::BigFloat) = p*(1/a)

function +(p::Poly, a::BigFloat)
    coeffs = copy(p.coeffs)
    coeffs[1] += a
    Poly(coeffs, p.cr)
end

(+)(a::BigFloat, p::Poly) = p+a
(-)(p::Poly, a::BigFloat) = p+(-a)

(+)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs+q.coeffs, p.cr) : error("!")
(-)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs-q.coeffs, p.cr) : error("!")

(*)(a::Array{BigFloat, 1}, b::BigFloat) = [ak*b for ak in a]
(*)(b::BigFloat, a::Array{BigFloat, 1}) = a*b

(/)(a::Array{BigFloat, 1}, b::BigFloat) = [ak/b for ak in a]

function *(p::Poly, q::Poly)
    if p.cr != q.cr error("!") end
    coeffs = fill(BigFloat(0), N+1)
    for j in 0:N
        for k in 0:N
            if j+k <= N
                coeffs[1+j+k] += p.coeffs[1+j]*q.coeffs[1+k]
            end
        end
    end
    Poly(coeffs, p.cr)
end

basis_element(p::Poly, k::Integer) = Poly([j==k ? BigFloat(1) : BigFloat(0) for j in 0:N], p.cr)

function identity(p::Poly)
    c, r = p.cr
    coeffs = fill(BigFloat(0), N+1)
    coeffs[1] = c
    coeffs[2] = r
    Poly(coeffs, p.cr)
end

function diff(p::Poly)
    c, r = p.cr
    Poly([k == N+1 ? BigFloat(0) : k*p.coeffs[1+k] for k in 1:N+1], p.cr)/r
end

# Power
function russian(t, p::Integer)
    if p == 0
        return one(t)
    elseif p == 1
        return t
    end
    
    # ignore trailing zeros in binary expansion
    trailing = trailing_zeros(p) + 1
    p >>= trailing
    
    # repeatedly square
    while (trailing -= 1) > 0
        t *= t
    end
    
    # compute the rest
    u = t
    while p > 0
        trailing = trailing_zeros(p) + 1
        p >>= trailing
        while (trailing -= 1) >= 0
            t *= t
        end
        u *= t
    end
    u
end

(^)(t::Poly, p::Integer) = russian(t, p)

# Evaluation and composition of polynomial.
function (t::Poly)(x::Union{BigFloat, Poly, Complex{BigFloat}})
    c, r = t.cr
    u = (x-c)/r
    total = t.coeffs[N+1]*u + t.coeffs[N]
    for k in N-1:-1:1
        total = total*u + t.coeffs[k]
    end
    total
end

(t::Poly)(x::Number) = t(BigFloat(x))
    
end

In [6]:
@everywhere begin

coeffs = fill(BigFloat(0), N+1)
mu_infty = BigFloat("-1.5949013562288205644978287")
coeffs[1:2] = [BigFloat(1), mu_infty]
p0 = Poly(coeffs, (BigFloat(0), BigFloat(1)))

zero_Omega = zero(Poly(fill(BigFloat(0), N+1), (c, r)))
I_Omega = identity(zero_Omega)

g0 = p0(I_Omega)
    
end

In [7]:
@everywhere poly_basis = [basis_element(g0, k) for k in 0:N];

In [8]:
@everywhere begin

Q(z) = z^d
Qdash(z) = d == 2 ? 2*z : d*z^(d-1)

function T(f)
    x = identity(f)
    a = f(1)
    (1/a)*f(Q(f(x*Q(a))))
end

function T(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
end

function T_and_DT(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
    
    g_dash = diff(g)
    g_dashxQa = g_dash(xQa)
    g_dashQgxQa = g_dash(QgxQa)
    
    dgxQadm1 = d*gxQa^(d-1)
    
    # we should ensure that any subexpressions
    # that do not depend on delta_g
    # are computed in the outer function, here.
    
    function DTg(delta_g)
        delta_a = delta_g(1)
        delta_Qa = d*a^(d-1)*delta_a
        delta_xQa = x*delta_Qa
        delta_gxQa = delta_g(xQa) + g_dashxQa*delta_xQa
        delta_QgxQa = dgxQadm1*delta_gxQa
        delta_gQgxQa = delta_g(QgxQa) + g_dashQgxQa*delta_QgxQa
        delta_Tg = (-1/a^2)*delta_a*gQgxQa + (1/a)*delta_gQgxQa
    end
    
    Tg, DTg
end
    
end

In [9]:
@everywhere begin

function eye(T::Type, n::Integer)
    I::Array{T, 2} = [j==k ? T(1) : T(0) for k in 1:n, j in 1:n]
end

# be careful that matrix_elements is only used from 1 process.
# pmap can then be used below.
# for large degree, this should help significantly.
# when the function being applied is a closure,
# there might be a benefit to using CachingPool(workers())
# as an additional, second, argument to pmap.
# actually, here, DTp is not a closure...
    
function matrix_elements(L, basis)
    J_polys = pmap(L, basis)
    #J_polys = pmap(L, CachingPool(workers()), basis)
    J = [J_polys[k].coeffs[j] for j in 1:N+1, k in 1:N+1]
end
    
function newtonstep(p::Poly)
    Tp, DTp = T_and_DT(p)
    Fp = Tp - p
    J = matrix_elements(DTp, poly_basis)
    I = eye(BigFloat, N+1)
    q = Poly(p.coeffs - inv(J-I)*Fp.coeffs, p.cr)
end
    
function newton(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1 = newtonstep(p0)
        error = abs(T(p1)-p1)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

## Skip if precomputed

In [ ]:
ga = g0
gb = T(ga)
erra = abs(gb-ga)
for k in 1:20
    gc = T(gb)
    errb = abs(gc - gb)
    println("$(k) $(errb)")
    if errb < erra
        gb, ga = gc, gb
        erra = errb
    else
        break
    end
end

In [ ]:
using Serialization
#open(f -> serialize(f, ga), "$(prefix)ga.jls", "w")
ga = open(deserialize, "$(prefix)ga.jls")

In [ ]:
ga.coeffs[1]

In [ ]:
abs(T(ga)-ga)

In [ ]:
#precompile
newton(g0, 0);

In [ ]:
@time gn = newton(ga, 20);

In [ ]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)g.jls", "w")

In [10]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)g.jls")
    
end

In [11]:
gn(0)

0.999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999352

In [12]:
gn(1)

-0.59160991663443815013962435438162895379022298919075582963905626080827016110024444655309687311596718431035214180064326974386372389312068288207799315961624092594115430529642761347098829399268704915779588874083761701454374048090852176811921141707111710425330824210097035806422600848343287080164784677856439804861554138904365

In [13]:
1/gn(1)

-1.69030297140524485334378015032416134822827805970956196668242326344973921908881055143276608578615291915193152630821259416410507756163090857294057319252627831020424401895602517765504793522623687664454132190710719267683493554697194567276686678514845145318901391119413556852821208047546969604755898739185932950666235922598764

In [14]:
residue_approx = T(gn)-gn
abs(residue_approx)

6.32219722834607033963127958555050817227207160942808246749377818873892277592547743648832460947555051186911220510108844238765938098243171742281710813202463745295957739260858335225717811192715666248511164047073478584307407816680050678113993057604933538847117566435556452351562572390108554439640939159160195139854339205565014e-320

In [15]:
ccc = gn.coeffs[1]

-0.000151171335061129165684026115927091901923775570970981721741458291806610434525924828836356997034390993684305294387095565163200728045636680125417549713449569423320469102252552574492161812762786829830685112885175097562816774077570383265530993215881160052371659054243871523164183598068583884702984027202906700261992146676290441

In [16]:
a = gn(1)
x = identity(gn)
xQa = x*Q(a)
psi(x) = (x-c)/r
norm1 = abs(psi(xQa))

0.75364232572374388642559314584525423132748397114787467033324286405808484824979727256100486193234202536287922460074048275410890290122269727045838556697346693657264436023656968240295609319639618159106220101257625513117221372274555702513440107551471256631555571167330186795168896776725745898011104718382320959599965091912712

In [17]:
QgxQa = Q(gn(xQa))
norm2 = abs(psi(QgxQa))

0.803456506833079315574654312297506629435330198722760835629229170742817646148571214296434581368027119437827172601921702663859697238320239970447291256156999052942497069063510529834442902560510264194885642373504199632451165391212698190484150813079245002854058125847568762187684022037360979944039653737159897852269175134422817

In [18]:
if norm1 >= 1 || norm2 >= 1
    println("Proof hopeless with these domains.")
end

In [ ]:
Tgn, DTgn = T_and_DT(gn)

In [ ]:
@time J = matrix_elements(DTgn, poly_basis);

In [ ]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)g.jls", "w")
open(f -> serialize(f, J), "$(prefix)jac.jls", "w")

## Continue...

In [19]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)g.jls")
J = open(deserialize, "$(prefix)jac.jls")
    
end

In [20]:
gn.cr == (c, r)

true

In [ ]:
Tgn, DTgn = T_and_DT(gn) # only if not done above

In [ ]:
for (k, a) in enumerate(gn.coeffs)
    println("$(k-1): $(a)")
end

In [21]:
a = gn(1)
alpha = 1/a

-1.69030297140524485334378015032416134822827805970956196668242326344973921908881055143276608578615291915193152630821259416410507756163090857294057319252627831020424401895602517765504793522623687664454132190710719267683493554697194567276686678514845145318901391119413556852821208047546969604755898739185932950666235922598764

In [22]:
@everywhere begin

# interval datatype
struct Interval
    lo::BigFloat
    hi::BigFloat
    Interval(lo, hi) = lo > hi ? error("bad Interval") : new(lo, hi)
    Interval(a) = Interval(a, a)
    Interval(i::Interval) = new(i.lo, i.hi)
end

# membership
function in(a::Number, x::Interval)
    x.lo <= a <= x.hi
end

# subset
function in(x::Interval, y::Interval)
    y.lo <= x.lo && x.hi <= y.hi
end

(==)(x::Interval, y::Interval) = x.lo == y.lo && x.hi == y.hi

(<)(x::Interval, y::Interval) = x.hi < y.lo
(>)(x::Interval, y::Interval) = y<x

(<)(x::BigFloat, y::Interval) = x < y.lo
(<)(x::Interval, y::BigFloat) = x.hi < y

(>)(x::BigFloat, y::Interval) = y<x
(>)(x::Interval, y::BigFloat) = y<x

zero(i::Interval) = Interval(0)
one(i::Interval) = Interval(1)

function abs(x::Interval)
    if 0 in x
        return Interval(0, max(abs(x.lo), abs(x.hi)))
    else
        return Interval(min(abs(x.lo), abs(x.hi)), max(abs(x.lo), abs(x.hi)))
    end
end

import Base: setrounding

function +(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+y.lo
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+y.hi
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(-)(x::Interval) = Interval(-x.hi, -x.lo)
(-)(x::Interval, y::Interval) = x+(-y)

function +(x::Interval, a::Number)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+a
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+a
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(+)(a::Number, x::Interval) = x+a
(-)(x::Interval, a::Number) = x+(-a)
(-)(a::Number, x::Interval) = -x+a

function *(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = min(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = max(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

function *(x::Interval, a::Number)
    if a >= 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.lo*a
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.hi*a
        setrounding(BigFloat, RoundNearest)
        Interval(c, d)
    else
        setrounding(BigFloat, RoundDown)
        e::BigFloat = x.hi*a
        setrounding(BigFloat, RoundUp)
        f::BigFloat = x.lo*a
        setrounding(BigFloat, RoundNearest)
        Interval(e, f)
    end
end

(*)(a::Number, x::Interval) = x*a

function iRec(x::Interval)
    if 0 in x
        error("division by zero")
    else
        setrounding(BigFloat, RoundDown)
        lo::BigFloat = 1/x.hi
        setrounding(BigFloat, RoundUp)
        hi::BigFloat = 1/x.lo
        setrounding(BigFloat, RoundNearest)
        Interval(lo, hi)
    end
end

(/)(x::Interval, y::Interval) = x*iRec(y)
(/)(x::Interval, a::Number) = x*(1/a)
(/)(a::Number, x::Interval) = a*iRec(x)

function ^(x::Interval, n::Integer)
    if n%2 == 1 || x.lo >= 0
        # odd power
        setrounding(BigFloat, RoundDown)
        a::BigFloat = x.lo^n
        setrounding(BigFloat, RoundUp)
        b::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(a, b)
    elseif x.hi < 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.hi^n
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.lo^n
        setrounding(BigFloat, RoundNearest)
        return Interval(c, d)
    else
        setrounding(BigFloat, RoundUp)
        e::BigFloat = x.lo^n
        f::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(0, max(e, f))
    end
end

function sqrt(x::Interval)
    half = BigFloat(1)/BigFloat(2)
    @assert half*BigFloat(2) == BigFloat(1)
    @assert x.lo >= BigFloat(0)
    setrounding(BigFloat, RoundDown)
    a = x.lo^half
    setrounding(BigFloat, RoundUp)
    b = x.hi^half
    setrounding(BigFloat, RoundNearest)
    Interval(a, b)
end

comparable(x::Interval, y::Interval) = x.hi <= y.lo || x.lo >= y.hi

setrounding(t::Type{Float64}, mode) = nothing
#setrounding(t::Type{Double64}, mode) = nothing

function bound_quasi_power_careful(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    g_norm = Interval(g_norm.hi, g_norm.hi)
    k = k0 + 1
    latest = k * g_norm ^ (k - 1)
    greatest_upper = latest.hi
    greatest_lower = latest.lo
    
    # a new upper bound strictly below a previous lower bound
    # indicates that we have passed the maximum.
    # at that point, the greatest upper bound found so far
    # and the greatest lower bound found so far
    # together bound the maximum of the graph.
    
    while latest.hi >= greatest_lower
        if latest.lo > greatest_lower
            greatest_lower = latest.lo
        end
        if latest.hi > greatest_upper
            greatest_upper = latest.hi
        end
        k = k + 1
        latest = k * g_norm ^ (k - 1)
        #println(latest)
    end
    Interval(greatest_lower, greatest_upper)
end

# here, we would make the argument that
# the rounded version of the quasi power
# is not order reversing at any point.

# this means that it should be sufficient to
# compute upper bounds only.

function bound_quasi_power(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    k = k0 + 1
    setrounding(BigFloat, RoundUp)
    latest = k * g_norm.hi ^ (k - 1)
    greatest_upper = latest
    while latest >= greatest_upper
        if latest > greatest_upper
            greatest_upper = latest
        end
        k = k + 1
        latest = k * g_norm.hi ^ (k - 1)
    end
    setrounding(BigFloat, RoundNearest)
    Interval(0, greatest_upper)
end

# we established that the two methods produce the same results
#for k in 1:100
#    p = Int(floor(rand()*500))
#    x = 1-0.001*rand()
#    bound1 = bound_quasi_power(p, Interval(x)).hi
#    bound2 = bound_quasi_power_crude(p, Interval(x)).hi
#    if bound1 != bound2
#        println("$x: $bound1 != $bound2")
#    end
#end

function balanced(i::Interval)
    a = max(abs(i.lo), abs(i.hi))
    Interval(-a, a)
end

function nonnegative(i::Interval)
    a = max(i.lo, BigFloat(0))
    @assert i.hi>=0
    Interval(a, i.hi)
end

function hull(v::Array{Interval, 1})
    lo = min([i.lo for i in v]...)
    hi = max([i.hi for i in v]...)
    Interval(lo, hi)
end

unit_vector(t::Type, k::Integer) = [j == k ? t(1) : t(0) for j in 0:N]
    
end

In [23]:
@everywhere begin

struct Rectangle
    re::Interval
    im::Interval
    Rectangle(re::Interval, im::Interval) = new(re, im)
    Rectangle(re, im) = new(Interval(re), Interval(im))
    Rectangle(re) = new(Interval(re), Interval(0))
end

in(x::Rectangle, z::Rectangle) = x.re in z.re && x.im in z.im
zero(z::Rectangle) = Rectangle(0, 0)
one(z::Rectangle) = Rectangle(1, 0)

(-)(z::Rectangle) = Rectangle(-z.re, -z.im)

(+)(z::Rectangle, w::Rectangle) = Rectangle(z.re+w.re, z.im+w.im)
(-)(z::Rectangle, w::Rectangle) = Rectangle(z.re-w.re, z.im-w.im)

(*)(z::Rectangle, x::Interval) = Rectangle(z.re*x, z.im*x)
(/)(z::Rectangle, x::Interval) = z*(1/x)

function reciprocal(z::Rectangle)
    d = z.re^2 + z.im^2
    Rectangle(z.re/d, -z.im/d)
end

(/)(z::Rectangle, w::Rectangle) = z*reciprocal(w)

(*)(z::Rectangle, w::Rectangle) = Rectangle(z.re*w.re-z.im*w.im, z.re*w.im+z.im*w.re)

(+)(z::Rectangle, x::Interval) = Rectangle(z.re+x, z.im)
(-)(z::Rectangle, x::Interval) = Rectangle(z.re-x, z.im)

# derived
(*)(x::Interval, z::Rectangle) = z*x
(/)(x::Interval, z::Rectangle) = Rectangle(x)/z
(+)(x::Interval, z::Rectangle) = z+x
(-)(x::Interval, z::Rectangle) = -z+x

# we don't have sqrt of an interval yet.
abs(z::Rectangle) = sqrt(z.re^2 + z.im^2)

(^)(z::Rectangle, p::Integer) = russian(z, p)

Rectangle(z::Complex{BigFloat}) = Rectangle(z.re, z.im)
    
end

In [24]:
@everywhere begin

struct Trunc
    coeffs::Array{Interval, 1}
    Trunc(a::Array{Interval, 1}) = length(a) == 1+N ? new(a) : error("wrong #coeffs")
end

abs(t::Trunc) = sum(abs(a) for a in t.coeffs)

(^)(t::Trunc, p::Integer) = russian(t, p)

zero(t::Trunc) = Trunc([Interval(0) for k in 0:N])
one(t::Trunc) = Trunc([k == 0 ? Interval(1) : Interval(0) for k in 0:N])

# Vector space essentials
(+)(t::Trunc, u::Trunc) = Trunc(t.coeffs + u.coeffs)
(-)(t::Trunc) = Trunc(-t.coeffs)
(*)(t::Trunc, a::Union{Number, Interval}) = Trunc(t.coeffs * a)

# Vector space derived
(-)(t::Trunc, u::Trunc) = t+(-u)
(/)(t::Trunc, a::Interval) = t*(1/a)
(*)(a::Union{Number, Interval}, t::Trunc) = t*a

# I think that (*)(Array{T, 1}, Number) is defined already by default.
# But since Interval is not a subtype of Number,
# we must define (*)(Array{Interval, 1}, Interval).
(*)(a::Array{Interval, 1}, b::Interval) = [ak*b for ak in a]
(*)(b::Interval, a::Array{Interval, 1}) = a*b

# Convenience
function +(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] += a
    Trunc(coeffs)
end

function -(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] -= a
    Trunc(coeffs)
end

(+)(a::Interval, t::Trunc) = t+a
(-)(a::Interval, t::Trunc) = (-t)+a

# Algebra
function *(t::Trunc, u::Trunc)
    coeffs = fill(Interval(0), N+1)
    for j in 0:N
        for k in 0:N-j
            coeffs[1+j+k] += t.coeffs[1+j]*u.coeffs[1+k]
        end
    end
    Trunc(coeffs)
end

# Evaluation and composition of polynomial. Compact, but could be made more efficient?
#(t::Trunc)(x::Union{Interval, Rectangle, Trunc}) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::Union{Interval, Rectangle, Trunc})
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function bound_high_order_product(t::Trunc, u::Trunc)
    coeffs = Dict{Int64, Interval}()
    for j in 0:N
        for k in 0:N
            if j+k > N
                a::Interval = t.coeffs[1+j]*u.coeffs[1+k]
                if haskey(coeffs, j+k)
                    coeffs[j+k] += a
                else
                    coeffs[j+k] = a
                end
            end
        end
    end
    sum(abs(a) for (k, a) in coeffs)
end

abs_star(t::Trunc) = sum(abs(t.coeffs[1+k]) for k in 1:N)

diff(t::Trunc) = Trunc([k == N ? Interval(0) : (1+k)*t.coeffs[2+k] for k in 0:N])

end

In [25]:
@everywhere begin

struct StdBall
    P::Trunc
    H::Interval
    G::Interval
    
    # we enforce the intervals h, g to be of the form [0, b].
    StdBall(p::Trunc, h::Number, g::Number) = new(p, Interval(0, h), Interval(0, g))
    StdBall(p::Trunc, h::Interval, g::Interval) = new(p, Interval(0, h.hi), Interval(0, g.hi))
end

zero(b::StdBall) = StdBall(zero(b.P), Interval(0), Interval(0))
one(b::StdBall) = StdBall(one(b.P), Interval(0), Interval(0))

# Abs
abs(b::StdBall) = nonnegative(abs(b.P) + b.H + balanced(b.G))

# Vector space operations

# Negative
(-)(b::StdBall) = StdBall(-b.P, b.H, b.G)

# Scalar multiples
(*)(b::StdBall, i::Interval) = StdBall(b.P*i, b.H*abs(i), b.G*abs(i))
(/)(b::StdBall, i::Interval) = b*(1/i)

# Add and subtract
(+)(b::StdBall, c::StdBall) = StdBall(b.P+c.P, b.H+c.H, b.G+c.G)
(-)(b::StdBall, c::StdBall) = b+(-c)

# Constant add and subtract
(+)(b::StdBall, i::Interval) = StdBall(b.P+i, b.H, b.G)
(-)(b::StdBall, i::Interval) = StdBall(b.P-i, b.H, b.G)

# Derived
(*)(i::Interval, b::StdBall) = b*i
(+)(i::Interval, b::StdBall) = b+i
(-)(i::Interval, b::StdBall) = (-b)+i

function *(b::StdBall, c::StdBall)
    PP_P = b.P*c.P
    PP_H = bound_high_order_product(b.P, c.P)
    PH = abs(b.P)*c.H
    PG = abs(b.P)*c.G

    HP = b.H*abs(c.P)
    HH = b.H*c.H
    HG = b.H*c.G
    
    GP = b.G*abs(c.P)
    GH = b.G*c.H
    GG = b.G*c.G
    
    P = PP_P
    H = PP_H + PH + HP + HH + GH + HG
    G = PG + GP + GG
    
    StdBall(P, H, G)
end

(^)(t::StdBall, p::Integer) = russian(t, p)

function abs_star(g::StdBall)
    x = abs_star(g.P) + Interval(0, g.G.hi) + Interval(-g.G.hi, g.G.hi)
    Interval(max(0, x.lo), x.hi)
end

# function ball compose

function (f::StdBall)(g::StdBall)
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    ball_1 = f.P(g)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G)
    g_norm = abs(g)^(N+1)
    g_star_norm = abs_star(g)^(N+1)
    ball_2 = StdBall(zero(f.P), f.H*g_star_norm, f.H*(g_norm - g_star_norm))
    ball_1 + ball_2 + ball_3
end

function (f::StdBall)(g) #could be g::AbstractRigorousScalar
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    g_norm = abs(g)^(N + 1)
    r = f.H * g_norm + f.G
    f.P(g) + Interval(-r.hi, r.hi)
end

# could absorb this into earlier definition
#(t::Trunc)(x::StdBall) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::StdBall)
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function diff_compose(f::StdBall, g::StdBall)
    if (f.H.hi > 0 || f.G.hi > 0) && ~(abs(g) in Interval(0, 1))
        error("diff composition not defined")
    end
    ball_1 = diff(f.P)(g)
    g_norm = abs(g)
    if f.G == f.H == Interval(0)
        norm_sup_H = norm_sup_G = 0
    else
        norm_sup_H = bound_quasi_power(N, g_norm)
        norm_sup_G = bound_quasi_power(0, g_norm)
    end
    # some sacrifice is made here! absorbing into general term!
    # an alternative is to absord into both high-order term and
    # the final polynomial coefficient.
    ball_2 = StdBall(zero(f.P), Interval(0), f.H * norm_sup_H)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G * norm_sup_G)
    ball_1 + ball_2 + ball_3
end

diff_compose(f::StdBall, t::Trunc) = diff_compose(f, StdBall(t, 0, 0))

function basis_element(b::StdBall, k::Integer)
    if k <= N
        return StdBall(Trunc(unit_vector(Interval, k)), 0, 0)
    else
        return StdBall(Trunc(fill(Interval(0), N+1)), 1, 0)
    end
end
    
end

In [26]:
@everywhere begin

struct Domain
    c::Interval
    r::Interval
    Domain(c::Interval, r::Interval) = r.lo <= 0 ? error("domain radius") : new(c, r)
end

in(z::Union{BigFloat, Interval}, d::Domain) = abs(z-d.c) < d.r
in(z::Rectangle, d::Domain) = abs(z-Rectangle(d.c, 0)) < d.r
(==)(d::Domain, e::Domain) = d.c==e.c && d.r==e.r
    
end

In [27]:
@everywhere begin

struct Ball
    std::StdBall
    dom::Domain
end

# negative
(-)(f::Ball) = Ball(-f.std, f.dom)

# scalar multiply
(*)(f::Ball, i::Interval) = Ball(f.std*i, f.dom)
(*)(i::Interval, f::Ball) = f*i
(/)(f::Ball, i::Interval) = f*(1/i)

(*)(f::Ball, a::Number) = f*Interval(a)
(*)(a::Number, f::Ball) = f*a

# ball add
(+)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std+g.std, f.dom) : error("domain mismatch")
(-)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std-g.std, f.dom) : error("domain mismatch")

# scalar add
(+)(f::Ball, a::Interval) = Ball(f.std+a, f.dom)
(-)(f::Ball, a::Interval) = Ball(f.std-a, f.dom)

# ball mul
(*)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std*g.std, f.dom) : error("domain mismatich")

# ball pow
(^)(f::Ball, n::Integer) = Ball(f.std^n, f.dom)

# ball eval/compose
(f::Ball)(i) = f.std((i - f.dom.c)/f.dom.r)
(f::Ball)(g::Ball) = Ball(f.std((g.std - f.dom.c)/f.dom.r), g.dom)

# diff_compose
diff_compose(f::Ball, g::Ball) = Ball(diff_compose(f.std, (g.std - f.dom.c)/f.dom.r)/f.dom.r, g.dom)

# allow differentiation for composition
function diff(f::Ball)
    df(g::Ball) = diff_compose(f, g)
end

# identity with respect to domain
function identity(f::Ball)
    coeffs = fill(Interval(0), N+1)
    coeffs[1] = f.dom.c
    coeffs[2] = f.dom.r
    Ball(StdBall(Trunc(coeffs), Interval(0), Interval(0)), f.dom)
end

basis_element(f::Ball, k::Integer) = Ball(basis_element(f.std, k), f.dom)

function Ball(p::Poly, h::Number, g::Number)
    c, r = p.cr
    dom = Domain(Interval(c), Interval(r))
    t = Trunc(map(Interval, p.coeffs))
    s = StdBall(t, h, g)
    b = Ball(s, dom)
end

abs(b::Ball) = abs(b.std)

function (m::Array{Interval, 2})(b::Ball, coeff_HH::Interval)
    PP = m*b.std.P.coeffs
    # assume all of G is in the polynomial part
    PG = [hull(m[1+k,:])*balanced(b.std.G) for k in 0:N]
    P = Trunc(PP + PG)
    H = (b.std.H + b.std.G)*abs(coeff_HH)
    G = Interval(0)
    Ball(StdBall(P, H, G), b.dom)
end
    
end

In [28]:
@everywhere dom = Domain(Interval(c), Interval(r))

In [29]:
@everywhere I = eye(BigFloat, N+1);

In [ ]:
@everywhere Lambda_pp = map(Interval, inv(J-I))

In [30]:
@everywhere begin

function gauss(A_::Matrix)
    A = copy(A_)      
    m, n = size(A)
    @assert m <= n

    for k in 1:m
        
        # seek largest nonzero A[j,k] across j
        best, abs_best = k, abs(A[k,k])
        for i in k+1:m
            if abs(A[i,k]) > abs_best
                best, abs_best = i, abs(A[i,k])
            end
        end

        # swap rows to position pivot
        A[k,:], A[best,:] = A[best,:], A[k,:]

        # divide row by pivot
        pivot = A[k,k]
        for j in k:n
            A[k,j] /= pivot
        end

        # clear zeros above and below
        for k_that in 1:m
            if k_that != k
                ratio = A[k_that,k]/A[k,k]
                for j in k:n
                    A[k_that,j] -= ratio*A[k,j]
                end
            end
        end
    end
    A
end

#invert(A::Matrix) = gauss(hcat(A, eye(typeof(A[1,1]), size(A)[1])))[:,size(A)[2]+1:end]
    
end

In [ ]:
# success here means that Lambda_pp, and hence Lambda itself,
# is invertible.
#@everywhere Lambda_pp_inv = invert(Lambda_pp);

In [ ]:
@everywhere function Phi(b0::Ball, Lambda_pp::Array{Interval, 2})
    Tb0, DTb0 = T_and_DT(b0)
    Phib0 = b0 - Lambda_pp(Tb0-b0, Interval(1))
end

In [ ]:
B0 = Ball(gn, 0, 0);

In [ ]:
@time Phi_B0 = Phi(B0, Lambda_pp);

In [ ]:
epsilon = abs(Phi_B0-B0)

In [ ]:
rho_log10 = Int(round(0.5+log10(epsilon.hi)))
rho = BigFloat(10)^rho_log10

In [ ]:
B1 = Ball(gn, 0, rho);

In [ ]:
open(f -> serialize(f, B0), "$(prefix)b0.jls", "w")
open(f -> serialize(f, B1), "$(prefix)b1.jls", "w")

In [31]:
@everywhere begin
    
B0 = open(deserialize, "$(prefix)b0.jls")
B1 = open(deserialize, "$(prefix)b1.jls")
rho = B1.std.G.hi
    
end

In [32]:
@everywhere psi(x) = (x-B1.dom.c)/B1.dom.r
a = B1(Interval(1))

Interval(-0.591609916634438150139624354381628953790222989190755829639056260808270161100244446553096873115967184310352141800643269743863723893120682882077993159616240925941154305296427613470988293992687049157795888740837617014543740480908521768119211417071117114253308242100970358064226008483432870801647846778564398048615541389043671, -0.591609916634438150139624354381628953790222989190755829639056260808270161100244446553096873115967184310352141800643269743863723893120682882077993159616240925941154305296427613470988293992687049157795888740837617014543740480908521768119211417071117094253308242100970358064226008483432870801647846778564398048615541389043629)

In [ ]:
@everywhere begin
T_B1, DT_B1 = T_and_DT(B1)
end

In [ ]:
dB_H = basis_element(B1, N+1)
DPhi_B1_dB_H = -Lambda_pp(DT_B1(dB_H), Interval(1))
norm_H = abs(DPhi_B1_dB_H)

In [33]:
@everywhere E = [basis_element(B1, k) for k in 0:N+1];

In [ ]:
# Precompute DT(B^1)(E_k) for all basis vectors, E_k.
@time DT_B1_E = pmap(DT_B1, E);

In [ ]:
open(f -> serialize(f, DT_B1_E), "$(prefix)DT_B1_E.jls", "w")

In [34]:
@everywhere begin

DT_B1_E = open(deserialize, "$(prefix)DT_B1_E.jls")
    
function bound_kappa_naively(k::Int)
    DPhi_B1_E_k = E[k] - Lambda_pp(DT_B1_E[k]-E[k], Interval(1))
    norm = abs(DPhi_B1_E_k)
end

end

In [ ]:
@time norms = pmap(bound_kappa_naively, [k for k in 1:N+2])

In [ ]:
norms[end] = norm_H
kappa = hull(norms).hi

In [ ]:
Phi_is_contractive = kappa < BigFloat(1)

In [ ]:
fixed_point_exists = epsilon < rho*(Interval(1)-kappa)

In [ ]:
println("Parameters:")
println("Power                     = $d")
println("Domain centre             = $c")
println("Domain radius             = $r")
println("Term G^u_0 (approx)       = $(gn.coeffs[1])")
println("Degree (reduced)          = $N")
println("Degree (original)         = $(N*d)")
println("Floating point type       = $(BigFloat)")
println("Precision (digits approx) = $P")
println("Precision (bits)          = $(precision(BigFloat))")
println()
println("Bounds:")
println("epsilon       = $(epsilon.hi)")
println("rho           = $rho")
#println("composition1  = $(norm1.hi)")
#println("composition2  = $(norm2.hi)")
println("kappa         = $kappa.hi")
println("rho*(1-kappa) = $((rho*(Interval(1)-kappa)).lo)")
println()
println("Conclusions:")
#println("proved domain extension   = $domain_extension")
println("proved phi is contractive = $Phi_is_contractive")
println("proved fixed point exists = $fixed_point_exists")
if fixed_point_exists
    println()
    println("Constants:")
    println("a = g(1)       = $(B1(1))")
    println("alpha = 1/g(1) = $(1/B1(1))")
end
println("Processors: 1+$(length(procs))")

In [ ]:
open(f -> serialize(f, [epsilon, rho, kappa]), "$(prefix)erk.jls", "w")

In [ ]:
# now we must decide what to do with the high-order
# and general bounds we could take each general
# bound [0, g] and add [-g, g] to every polynomial
# term for the high-order bound [0, h] we form
# [-h, h]+[-g, g].  in this way we form what i call
# the augmented matrix $M$

M_aug = [Interval(0) for j in 1:N+2, k in 1:N+2]

for k in 1:N+2
    b_H = balanced(DT_B1_E[k].std.H)
    b_G = balanced(DT_B1_E[k].std.G)
    for j in 1:N+1
        M_aug[j,k] = DT_B1_E[k].std.P.coeffs[j] + b_G
    end
    M_aug[N+2,k] = b_H + b_G
end
M_aug;

# ignoring the high-order parts yields the so-called PP matrix
M_PP = M_aug[1:N+1,1:N+1];

In [ ]:
# we now find approximate eigvenvalues and vectors of the PP matrix

In [ ]:
using GenericSchur

M_approx = map(i -> i.lo, M_PP)
@time vals_rev, vecs_rev = GenericSchur.eigen(M_approx);

In [ ]:
vals = reverse(vals_rev)

In [ ]:
open(f -> serialize(f, vals), "$(prefix)val.jls", "w")

In [ ]:
vecs_rev_re = map(x->BigFloat(x), vecs_rev)
vecs = vecs_rev_re[1:N+1,N+1:-1:1];

In [ ]:
normalised_T = [vecs[:,k]/sum(map(abs, vecs[:,k])) for k in 1:N+1]
vecs = [normalised_T[k][j] for j in 1:N+1, k in 1:N+1];

In [ ]:
eigen_error = sum(map(abs, M_PP*vecs[:,1] - vals[1]*vecs[:,1]))

In [35]:
@everywhere begin

span(i::Interval) = Interval(0, (i-i).hi)

phi(p::Poly) = p.coeffs[1]
phi(b::Ball) = b.std.P.coeffs[1] + balanced(b.std.G)
    
end

In [ ]:
phi(B0)

In [ ]:
span(phi(B0))

In [ ]:
phi(B1)

In [ ]:
span(phi(B1))

In [ ]:
i = 2
val_approx = vals[i]
vec_approx = vecs[1:end,i]

# normalize properly
f0_poly_coeffs = vec_approx/vec_approx[1]*val_approx.re
f0_poly_coeffs[1]

In [ ]:
f0_poly = Poly(f0_poly_coeffs, (c, r))

In [ ]:
abs(DTgn(f0_poly)-f0_poly*vals[i].re)

#computation dies here.
#error is at the minus, but
#likely the error is DTgn not being in scope?
#but this is non-rigorous check
#however we need DTgn later?

#the error appears to be from - domain mismatch.
#let's check the domains next.

In [36]:
@everywhere begin
    
function newtonstep2(p::Poly)
    Fp = DTgn(p) - p.coeffs[1]*p
    Z = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
    for j in 1:N+1
        for k in 1:N+1
            if k == 1
                Z[j, k] += p.coeffs[j]
            end
            if j == k
                Z[j, k] += p.coeffs[1]
            end
        end
    end
    q = Poly(p.coeffs - inv(J-Z)*Fp.coeffs, p.cr)
    q, abs(Fp)
end

function newton2(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep2(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

In [ ]:
@time fn = newton2(f0_poly, 20);

In [ ]:
@time Delta_PP = matrix_elements(DTgn, poly_basis);

In [ ]:
Delta_mod = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
for j in 1:N+1
    for k in 1:N+1
        Delta_mod[j,k] = Delta_PP[j,k]
        if k == 1
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[j]
        end
        if j == k
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[1]
        end
    end
end
Delta_mod;

In [ ]:
L_pp = map(Interval, inv(Delta_mod));

# now we check that the resulting interval matrix is invertible
#@time invert(L_pp);

In [ ]:
V0 = Ball(StdBall(Trunc(map(Interval, fn.coeffs)), BigFloat(0), BigFloat(0)), dom);

In [ ]:
eigenfunction_error = abs(V0*phi(V0) - DT_B1(V0))

In [ ]:
V0.dom == B1.dom

In [ ]:
open(f -> serialize(f, L_pp), "$(prefix)L_pp.jls", "w")
open(f -> serialize(f, V0), "$(prefix)V0.jls", "w")

In [37]:
@everywhere begin

L_pp = open(deserialize, "$(prefix)L_pp.jls")
V0 = open(deserialize, "$(prefix)V0.jls")
    
function Psi(v::Ball, L_pp::Array{Interval, 2})
    Psi_v = v - L_pp(DT_B1(v)-v*phi(v), -Interval(1)/V0.std.P.coeffs[1])
end
    
end

In [ ]:
@time epsilon_hat = abs(Psi(V0, L_pp)-V0)

In [ ]:
rho_hat = BigFloat(10)^Int(round(0.5+log10(epsilon_hat.hi)))

In [ ]:
V1 = Ball(fn, 0, rho_hat);

In [ ]:
open(f -> serialize(f, V1), "$(prefix)V1.jls", "w")

In [ ]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

In [ ]:
rho_hat

In [38]:
@everywhere begin
    
V1 = open(deserialize, "$(prefix)V1.jls")

function DPsi(b::Ball, L_pp::Array{Interval, 2})
    function DPsi_b(db::Ball)
        DF = DTb1(db)-phi(b)*db-phi(db)*b
        DPsi_b_db = db - L_pp(DF, -Interval(1)/phi(V0))
    end
    DPsi_b
end

function bound_kappa_hat_naively(k::Int)
    b = V1
    DPsi_b_es_k = E[k] - L_pp(DT_B1_E[k]-phi(b)*E[k]-phi(E[k])*b, -Interval(1)/phi(V0))
    norm = abs(DPsi_b_es_k)
end
    
end

In [ ]:
EH = basis_element(V1, N+1)
DT_G_EH = DT_B1(EH)
DPsi_V1_EH = (Interval(1)-phi(V1)/phi(V0))*EH -L_pp(DT_G_EH, -Interval(1)/phi(V0))
norm_H = abs(DPsi_V1_EH)

In [ ]:
@time norms_v = pmap(bound_kappa_hat_naively, [k for k in 1:N+2])

In [ ]:
norms_v[end] = norm_H
kappa_hat = hull(norms_v)

In [ ]:
epsilon_hat < rho_hat*(Interval(1)-kappa_hat)

In [ ]:
epsilon_hat

In [ ]:
rho_hat*(Interval(1)-kappa_hat)

In [ ]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

# Critical scaling of additive noise

In [39]:
@everywhere function make_noise_functions(G)
    
    a = G(1)
    
    X = identity(G)
    XQa = X*Q(a)
    GXQa = G(XQa)
    QGXQa = Q(GXQa)
    GQGXQa = G(QGXQa)
    
    Gdash = diff(G)
    GdashQa = Gdash(XQa)
    GdashQGXQa = Gdash(QGXQa)
    
    t2 = (1/a^2)
    t3 = ((1/a)*GdashQGXQa*Qdash(GXQa))^2
    
    function T_noise(H)
        term2 = t2*H(QGXQa)
        term3 = t3*H(XQa)
        lhs = term2 + term3
        return lhs
    end

    function DT_noise(H)
        function DT_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            return lhs
        end
        return DT_noise_H
    end

    function F_noise(H)
        lhs = T_noise(H)
        rhs = phi(H)^2*H
        return lhs - rhs
    end
    
    function DF_noise(H)
        function DF_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            rhs = 2*phi(H)*phi(K)*H + phi(H)^2*K
            return lhs - rhs
        end
        return DF_noise_H
    end
    
    return T_noise, DT_noise, F_noise, DF_noise
end

In [40]:
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(gn)

(T_noise, DT_noise, F_noise, DF_noise)

In [41]:
function newtonstep3(p::Poly)
    F_noise_p = F_noise(p)
    
    DF_noise_p = DF_noise(p)
    Gamma_noise_inv = matrix_elements(DF_noise_p, poly_basis);
    
    q = Poly(p.coeffs - inv(Gamma_noise_inv)*F_noise_p.coeffs, p.cr)
    q, abs(F_noise_p)
end

function newton3(p0::Poly, n::Integer)
    basis = [basis_element(p0, k) for k in 0:N]
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep3(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end

newton3 (generic function with 1 method)

In [42]:
v_noise_coeffs = BigFloat[8.2439108542525868183984625308166, -6.0314979598335030543896628404458, -0.99906557440532477623573316007567, 2.6365397214358035500680010442941, -0.81393255099831777701118943916041, -0.2916501997294583937683387447899, 0.27455204544220932923048986000544, -0.04629827836891161318089224352731, -0.03003398317265670879893203510508, 0.017474018792551148952490914683149]

10-element Array{BigFloat,1}:
  8.243910854252586517532108700834214687347412109375         
 -6.0314979598335032306977154803462326526641845703125        
 -0.9990655744053247300229259053594432771205902099609375     
  2.63653972143580350717684268602170050144195556640625       
 -0.8139325509983177386885699888807721436023712158203125     
 -0.29165019972945838144795516200247220695018768310546875    
  0.27455204544220934703702141632675193250179290771484375    
 -0.046298278368911614799596776492762728594243526458740234375
 -0.03003398317265670858233761464362032711505889892578125    
  0.017474018792551150591751962792841368354856967926025390625

In [43]:
v_noise_0 = Poly([k <= length(v_noise_coeffs) ? v_noise_coeffs[k] : BigFloat(0) for k in 1:N+1], (c, r))
v_noise_0(0), phi(v_noise_0)

(10.9621005673414116003563562277573216712884357581514427277456736817273785788984241662546992301940917968749999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999, 8.243910854252586517532108700834214687347412109375)

In [44]:
v_noise_n = newton3(v_noise_0, 20);

1: 0.454837852395630709055392316170017424365370838090626425442378484986421392888101590157702013471394976248536888100966475027354248487049574149648397655044528568077420885254541326407061093557896689706439557584621156447035135178729270991428213998447054650659537584767416781363347972429224951044249083180787764848580046396221107
2: 3.30375516301673304367710704074673438310699703412850107608948296582697205803669644188977790017236431680902165408619024907697159348978324721724765452969188831928178219004392681682909830696128385020787988563150751759989399022598458659248161343420693223921448422444043791265480675346377056457809677909089152332250178404139053e-17
3: 2.47464309052560811945442001253620403794582969455250517501272320019516790910696459983913945253463871063253789409375715993564804624412909551582059869445457058795997827092343572250263201358797291862159687321914826567894854213825842821956999409427397990077091432937362174618583885578658236887746670079687003463349571225450018e-37
4: 2.33850743

In [45]:
open(f -> serialize(f, v_noise_n), "$(prefix)noise_n.jls", "w")

@everywhere v_noise_n = deserialize("$(prefix)noise_n.jls")

In [46]:
phi(v_noise_n)

8.24391085425258681839846236502923761606731776662405840926219256825653663924142562689964204720757842422300873689832234963510717328253743947119166688892324018278114543435570594770800377985238316683467659857290704875987642458476648182567707405595689842976849327088118449196788121462757670908015117705258032330416064898594114

In [47]:
@time Lambda_PP_inv_noise = matrix_elements(DF_noise(v_noise_n), poly_basis);

2183.053496 seconds (12.57 M allocations: 473.218 MiB, 0.01% gc time)


In [48]:
@time Lambda_PP_noise = map(Interval, inv(Lambda_PP_inv_noise));

 53.065379 seconds (593.41 M allocations: 57.462 GiB, 14.25% gc time)


In [49]:
open(f -> serialize(f, Lambda_PP_noise), "$(prefix)Lambda_PP_noise.jls", "w")

@everywhere begin
Lambda_PP_noise = deserialize("$(prefix)Lambda_PP_noise.jls");
end

In [50]:
#@time Lambda_PP_noise_inv_check = inv(Lambda_PP_noise);

In [ ]:
@everywhere begin
    
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(B1)

function Phi_noise(b::Ball)
    b - Lambda_PP_noise(F_noise(b), -1/phi(b)^2)
end
    
end

In [52]:
B0_noise = Ball(v_noise_n, BigFloat(0), BigFloat(0));

In [53]:
@time epsilon_noise = abs(Phi_noise(B0_noise) - B0_noise)

433.196371 seconds (7.59 G allocations: 705.015 GiB, 19.19% gc time)


Interval(0.0, 5.30557574029653007784375492872222769234313569654218565827762872972975002025774650106654475828710597747703149576883715412992750998704765691680361164189769116242423432507624004568985760578868515240629503039164942748038184785348621937116203639703900305288587604956458888013680004239999155548302794920602892262155167694942758e-242)

In [54]:
rho_noise = Interval(BigFloat(10)^Int(round(0.5+log10(epsilon_noise.hi))))

Interval(1.0e-241, 1.0e-241)

In [55]:
B1_noise = Ball(v_noise_n, BigFloat(0), BigFloat(rho_noise.hi));

In [56]:
println(phi(B1_noise).lo)
println(phi(B1_noise).hi)
println(1+rho_noise.hi)

8.24391085425258681839846236502923761606731776662405840926219256825653663924142562689964204720757842422300873689832234963510717328253743947119166688892324018278114543435570594770800377985238316683467659857290704875987642458476648182567707405585689842976849327088118449196788121462757670908015117705258032330416064898594108
8.24391085425258681839846236502923761606731776662405840926219256825653663924142562689964204720757842422300873689832234963510717328253743947119166688892324018278114543435570594770800377985238316683467659857290704875987642458476648182567707405605689842976849327088118449196788121462757670908015117705258032330416064898594119
1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000009999999999999999999999999999999999999999999999999999999999999999999999999999999


In [57]:
open(f -> serialize(f, B0_noise), "$(prefix)B0_noise.jls", "w")
open(f -> serialize(f, B1_noise), "$(prefix)B1_noise.jls", "w")

@everywhere begin
B0_noise = deserialize("$(prefix)B0_noise.jls");
B1_noise = deserialize("$(prefix)B1_noise.jls");
end

In [58]:
@everywhere begin
    
function DPhi_noise(b::Ball, L_pp::Array{Interval, 2})
    DF_noise_b = DF_noise(b)
    function DPhi_noise_b(db::Ball)
        DF = DF_noise_b(db)
        DPhi_b_db = db - L_pp(DF, -Interval(1)/phi(V0)^2)
    end
    DPhi_noise_b
end

function bound_kappa_noise_naively(k::Int)
    b = B1_noise
    L_pp = Lambda_PP_noise
    DPhi_noise_b = DPhi_noise(b, L_pp)
    DPhi_noise_b_es_k = DPhi_noise_b(E[k])
    norm = abs(DPhi_noise_b_es_k)
end
    
end

In [59]:
EH = basis_element(B1_noise, N+1)
DT_noise_B1_EH = DT_noise(B1_noise)(EH)
DPhi_noise_B1_EH = (Interval(1)-(phi(B1_noise)/phi(B0_noise))^2)*EH -L_pp(DT_noise_B1_EH, -Interval(1)/phi(V0))
norm_H = abs(DPhi_noise_B1_EH)

Interval(0.0, 4.11603001660192150912262828033891699369877343274731758217205842965092270960795382157606742460435674595096983184564436792338294237547367356868933076758968100756781962192456798130309735452547136172699062278332625024596857037023423776015658939202547805600349384525301983491910973523583435719073635260467547139204683951998967e-45)

In [60]:
@time norms_w = pmap(bound_kappa_noise_naively, [k for k in 1:N+2])

52360.993610 seconds (2.80 M allocations: 104.160 MiB, 0.00% gc time)


482-element Array{Interval,1}:
 Interval(0.0, 1.18115183282446060742769803269497548421234404719522011414659507982044936060521492239012324837523458452162956400176245165029426078079018015771805997124048718227440883539187174453006772977694198107921369899994823698958489346304542130011294776047471177597758923884132194582011715311829103604551583248938391736311653771710248e-239)
 Interval(0.0, 4.74805713348789855699451703151566532478317923409790955208346420440041815085380545425995762496988202784886345241484713604844105875339851983172553159316821041371972047374030466793012663442167688751310217375028152461711552331539173992925521186805562220749057296065836738528883597731927185809885538678886989398495142560618779e-242)
 Interval(0.0, 4.616766033105827265234282576109574570682031932853594810775665815189241130965453448560584285049039216348167368205495145580081464149028558685220212867939746572999388834957340647317228902646942817646233946996917068118026785693513304241783607594764361061090393327037466

In [61]:
norms_w[end] = norm_H
kappa_noise = Interval(hull(norms_w).hi)

Interval(2.31794121734352752533583599905761850914777601659795338099602813993329039259251901282908395244688029775594617143556539835584716650453138679624603115280827638871317991145709562634544360471830567211942881891057027395896313658224806865023580883759625115507965100364033015842296650353002436148001309109975120645042428943342176e-37, 2.31794121734352752533583599905761850914777601659795338099602813993329039259251901282908395244688029775594617143556539835584716650453138679624603115280827638871317991145709562634544360471830567211942881891057027395896313658224806865023580883759625115507965100364033015842296650353002436148001309109975120645042428943342176e-37)

In [62]:
noise_existence = epsilon_noise < rho_noise*(Interval(1)-kappa_noise)

true

In [66]:
println("Degree of map at critical point, d:\n$(d)")
println("Domain centre, c:\n$(c)")
println("Domain radius, r:\n$(r)")
println("Truncation degree, N:\n$(N)")
println("Precision (digits approx.), P:\n$(P)")

#println("\nepsilon:\n$(epsilon.hi)")
#println("rho:\n$(rho)")
#println("kappa:\n$(kappa)")

#println("\nepsilon_hat:\n$(epsilon_hat.hi)")
#println("rho_hat:\n$(rho_hat)")
#println("kappa_hat:\n$(kappa_hat.hi)")

println("\nepsilon_noise:\n$(epsilon_noise.hi)")
println("rho_noise:\n$(rho_noise.hi)")
println("kappa_noise:\n$(kappa_noise.hi)")

println("\na=G(1) lower bound:\n$(B1(1).lo)")
println("a=G(1) upper bound:\n$(B1(1).hi)")
println("1+rho:\n$(1+rho)")

println("\nalpha=1/G(1) lower bound:\n$((1/B1(1)).lo)")
println("alpha=1/G(1) upper bound:\n$((1/B1(1)).hi)")
println("1+rho:\n$(1+rho)")

#println("\ndelta=phi(V) lower bound:\n$(phi(V1).lo)")
#println("delta=phi(V) upper bound:\n$(phi(V1).hi)")
#println("1+rho_hat:\n$(1+rho_hat)")

println("\ngamma=phi(K) lower bound:\n$(phi(B1_noise).lo)")
println("gamma=phi(K) upper bound:\n$(phi(B1_noise).hi)")
println("1+rho_noise:\n$(1+rho_noise.hi)")

Degree of map at critical point, d:
4
Domain centre, c:
0.575400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001
Domain radius, r:
0.799999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996
Truncation degree, N:
480
Precision (digits approx.), P:
320

epsilon_noise:
5.3055757402965300778437549287222276923431356965421856582776287297297500202577465010665447582871059774770314957688371541299275099870476569168036116418976911624242343250762400456898576057886851524062950

In [67]:
rmprocs(procs)

Task (done) @0x00007eff26091390